In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import mean_absolute_error as MAE
from scipy import stats
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import metrics
import numpy as np
from gensim.models import word2vec
from gensim.corpora.dictionary import Dictionary
import pickle
from keras.preprocessing import sequence

import time, datetime
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve
import gc
from sklearn.model_selection import GridSearchCV
import jieba, pdb
from sklearn.decomposition import PCA


jieba.set_dictionary('jieba_dict/dict.txt.big')
# load stopwords set
stopword_set = set()
with open('jieba_dict/stopwords.txt','r', encoding='utf-8') as stopwords:
    for stopword in stopwords:
        stopword_set.add(stopword.strip('\n'))

model = word2vec.Word2Vec.load("word2vec2.model")


def create_dictionaries(p_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(p_model.wv.vocab.keys(), allow_update=True)
    w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2indx.keys()}  # 词语的词向量
    return w2indx, w2vec

def Convert_orderid(x):
    return str(x).strip('\n')

def Convert_Date(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = pd.to_datetime(Year+'-'+Month+'-'+Day)
    return date1

def Date2Ticks(x):
    Year='20'+x[-2:]
    Month=month[x[-6:-3]]
    Day=x[:-7]
    date1 = str(Year+'/'+Month+'/'+Day)
    return time.mktime(datetime.datetime.strptime(date1, "%Y/%m/%d").timetuple())

index_dict, word_vectors= create_dictionaries(model)
output = open("wordwmbedding.pkl", 'wb')
pickle.dump(index_dict, output)  # 索引字典
pickle.dump(word_vectors, output)  # 词向量字典
output.close()



C:\Users\m1217\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.
C:\Users\m1217\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:43: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [3]:
df_train_1 = pd.read_csv("myTrain3.csv")
df_result_1 = pd.read_csv("myTest3.csv")


used_columns=['order_id','tick_diff', 'Source_1', 'Source_2', 'Unit', 'deal_or_not',
'people_amount', 'Begin_Tick','days', 'Order_Tick', 'Area', 'SubLine', 'price','PreDays','Begin_Date_Weekday', 
'Order_Date_Weekday', 'Return_Date_Weekday', 'fly_t', 'fly_date',
"src_airport", "arrive_t", "arrive_date", "dst_airport",'pred0','pred1','pred2','pred3','pred4','pred5','pred6', 'pred7','product_name']

df_train_1 = df_train_1[used_columns]
df_result_1 = df_result_1[used_columns]

myPred = []
for i in range(8):
    myPred.append(df_train_1['pred%d'%(i)].values.tolist())
myPred = np.asarray(myPred)
print(myPred.shape)
predFeat = []
for i in range(len(myPred[0,:])):
    predFeat.append([np.max(myPred[:,i]), np.percentile(myPred[:,i], 75), 
                     np.median(myPred[:,i]), np.percentile(myPred[:,i],25),np.min(myPred[:,i])]
                   )
train_feat=np.asarray(predFeat)

myPred = []
for i in range(8):
    myPred.append(df_result_1['pred%d'%(i)].values.tolist())
myPred = np.asarray(myPred)
predFeat = []
for i in range(len(myPred[0,:])):
    predFeat.append([np.max(myPred[:,i]), np.percentile(myPred[:,i], 75), 
                     np.median(myPred[:,i]), np.percentile(myPred[:,i],25),
                   np.min(myPred[:,i])])
test_feat=np.asarray(predFeat)

for i in range(8):
    del df_result_1['pred%d'%(i)]
    del df_train_1['pred%d'%(i)]

Y=df_train_1['deal_or_not'].values.tolist()
swX = (df_train_1['product_name']).values.tolist()
del df_train_1['deal_or_not'] 
del df_train_1['product_name']
del df_train_1['order_id']
X = df_train_1.values.tolist()

swrx = (df_result_1['product_name']).values.tolist()
rid = df_result_1['order_id'].values.tolist()
del df_result_1['product_name']
del df_result_1['deal_or_not'] 
del df_result_1['order_id'] 
rx = df_result_1.values.tolist()


(8, 296237)


In [4]:
sX, Y =np.asarray(X), np.asarray(Y)
X=np.concatenate([sX, train_feat], axis=1)
rx = np.asarray(rx)
rx=np.concatenate([rx, test_feat], axis=1)



def text_to_index_array(p_new_dic, p_sen):  # 文本转为索引数字模式
    new_sentences = []
    for sen in p_sen:
        new_sen = []
        for word in str(sen):
            try:
                new_sen.append(p_new_dic[word])  # 单词转索引数字
            except:
                new_sen.append(0)  # 索引字典里没有的词转为数字0
        new_sentences.append(new_sen)

    return np.array(new_sentences)


wX = text_to_index_array(index_dict, swX)
wrx = text_to_index_array(index_dict, swrx)

In [5]:
wX = sequence.pad_sequences(wX, maxlen=60)
wrx = sequence.pad_sequences(wrx, maxlen=60)


X=np.concatenate([X, wX], axis=1)
rx=np.concatenate([rx, wrx], axis=1)
# xlen=len(X)
# from sklearn.preprocessing import normalize
# Xtmp=normalize(np.concatenate([X, rx], axis=0),norm='max', axis=0)
# X=Xtmp[:xlen]
# rx=Xtmp[xlen:]

print(X.shape)
print(rx.shape)

print(X[5,-5:-1])

(296237, 86)
(99736, 86)
[8046. 8801.    0.    0.]


In [7]:

folds = StratifiedKFold(n_splits= 10, shuffle=True, random_state=666)

oof_preds = np.zeros(X.shape[0])
sub_preds = np.zeros(rx.shape[0])
feature_importance_df = pd.DataFrame()


for n_fold, (train_idx, valid_idx) in enumerate(folds.split(X, Y)):

    train_x, train_y = X[train_idx,:], Y[train_idx]
    valid_x, valid_y = X[valid_idx,:], Y[valid_idx]

    print("Train Index:",train_idx,",Val Index:",valid_idx)

    params = {
    'nthread': 8, 'boosting_type': 'dart','objective': 'regression', 'metric': 'auc', 
        'learning_rate': 0.01, 'num_leaves': 70,
    'max_depth': 10, 'subsample': 0.2, 'feature_fraction': 0.2, 
        'min_split_gain': 0.09, 'min_child_weight': 9.5,
        'min_data_in_leaf':10,
        'bagging_freq':5,
    'drop_rate':0.5, 'skip_drop':0.5, 'max_drop':5, 'uniform_drop':False, 
        'xgboost_dart_mode':True, 'drop_seed':1 }


    if n_fold >= 0:
        dtrain = lgb.Dataset(train_x, label=train_y)
        dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
        bst = lgb.train(params, dtrain, num_boost_round=10000, valid_sets=[dtrain,dval], early_stopping_rounds=200, verbose_eval=50)


        tmp_valid = bst.predict(valid_x, num_iteration=bst.best_iteration)
        oof_preds[valid_idx] = tmp_valid
        sub_preds += bst.predict(rx, num_iteration=bst.best_iteration) / folds.n_splits
        

        # Make the feature importance dataframe
        gain = bst.feature_importance('gain')
        fold_importance_df = pd.DataFrame({'feature':bst.feature_name(),'split':bst.feature_importance('split'),
                                           'gain':100*gain/gain.sum(),  'fold':n_fold,}).sort_values('gain',ascending=False)

        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))

        del bst, train_x, train_y, valid_x, valid_y
        

Train Index: [     0      1      2 ... 296233 296234 296236] ,Val Index: [    15     37     46 ... 296213 296215 296235]
Training until validation scores don't improve for 200 rounds.
[50]	training's auc: 0.672377	valid_1's auc: 0.659163
[100]	training's auc: 0.675841	valid_1's auc: 0.661932
[150]	training's auc: 0.678353	valid_1's auc: 0.662137
[200]	training's auc: 0.681063	valid_1's auc: 0.663636
[250]	training's auc: 0.683699	valid_1's auc: 0.665103
[300]	training's auc: 0.685877	valid_1's auc: 0.666259
[350]	training's auc: 0.688775	valid_1's auc: 0.667245
[400]	training's auc: 0.691481	valid_1's auc: 0.668121
[450]	training's auc: 0.694026	valid_1's auc: 0.669205
[500]	training's auc: 0.696192	valid_1's auc: 0.670145
[550]	training's auc: 0.69834	valid_1's auc: 0.670933
[600]	training's auc: 0.701232	valid_1's auc: 0.671754
[650]	training's auc: 0.703774	valid_1's auc: 0.67277
[700]	training's auc: 0.706593	valid_1's auc: 0.673647
[750]	training's auc: 0.708885	valid_1's auc: 0.6

[500]	training's auc: 0.697404	valid_1's auc: 0.666129
[550]	training's auc: 0.699721	valid_1's auc: 0.667002
[600]	training's auc: 0.70237	valid_1's auc: 0.668112
[650]	training's auc: 0.704858	valid_1's auc: 0.668816
[700]	training's auc: 0.707518	valid_1's auc: 0.669575
[750]	training's auc: 0.709907	valid_1's auc: 0.670494
[800]	training's auc: 0.711589	valid_1's auc: 0.670793
[850]	training's auc: 0.713914	valid_1's auc: 0.671644
[900]	training's auc: 0.716111	valid_1's auc: 0.672112
[950]	training's auc: 0.717748	valid_1's auc: 0.672484
[1000]	training's auc: 0.719386	valid_1's auc: 0.672733
[1050]	training's auc: 0.721478	valid_1's auc: 0.673088
[1100]	training's auc: 0.723362	valid_1's auc: 0.673635
[1150]	training's auc: 0.725295	valid_1's auc: 0.674157
[1200]	training's auc: 0.727001	valid_1's auc: 0.673952
[1250]	training's auc: 0.728537	valid_1's auc: 0.67412
[1300]	training's auc: 0.730095	valid_1's auc: 0.674198
[1350]	training's auc: 0.731568	valid_1's auc: 0.674308
[140

[1750]	training's auc: 0.74173	valid_1's auc: 0.689997
[1800]	training's auc: 0.743086	valid_1's auc: 0.690161
[1850]	training's auc: 0.744324	valid_1's auc: 0.690122
[1900]	training's auc: 0.745356	valid_1's auc: 0.690342
[1950]	training's auc: 0.746409	valid_1's auc: 0.69049
[2000]	training's auc: 0.747558	valid_1's auc: 0.690743
[2050]	training's auc: 0.748774	valid_1's auc: 0.690953
[2100]	training's auc: 0.750053	valid_1's auc: 0.691144
[2150]	training's auc: 0.751125	valid_1's auc: 0.691084
[2200]	training's auc: 0.752107	valid_1's auc: 0.690986
[2250]	training's auc: 0.753407	valid_1's auc: 0.691013
[2300]	training's auc: 0.754337	valid_1's auc: 0.691304
[2350]	training's auc: 0.755282	valid_1's auc: 0.691244
[2400]	training's auc: 0.756502	valid_1's auc: 0.691255
[2450]	training's auc: 0.757518	valid_1's auc: 0.691235
[2500]	training's auc: 0.758566	valid_1's auc: 0.691178
Early stopping, best iteration is:
[2318]	training's auc: 0.754595	valid_1's auc: 0.691353
Fold  5 AUC : 0

Fold  7 AUC : 0.688970
Train Index: [     0      2      3 ... 296234 296235 296236] ,Val Index: [     1     16     25 ... 296155 296206 296216]
Training until validation scores don't improve for 200 rounds.
[50]	training's auc: 0.671848	valid_1's auc: 0.664193
[100]	training's auc: 0.675645	valid_1's auc: 0.667083
[150]	training's auc: 0.678234	valid_1's auc: 0.667967
[200]	training's auc: 0.680812	valid_1's auc: 0.669467
[250]	training's auc: 0.683658	valid_1's auc: 0.670429
[300]	training's auc: 0.685741	valid_1's auc: 0.671127
[350]	training's auc: 0.688414	valid_1's auc: 0.67237
[400]	training's auc: 0.690987	valid_1's auc: 0.673486
[450]	training's auc: 0.693651	valid_1's auc: 0.674469
[500]	training's auc: 0.695872	valid_1's auc: 0.675528
[550]	training's auc: 0.698459	valid_1's auc: 0.676592
[600]	training's auc: 0.701078	valid_1's auc: 0.677691
[650]	training's auc: 0.703464	valid_1's auc: 0.678573
[700]	training's auc: 0.706002	valid_1's auc: 0.679713
[750]	training's auc: 0.7

[2850]	training's auc: 0.765432	valid_1's auc: 0.68349
[2900]	training's auc: 0.76635	valid_1's auc: 0.683636
[2950]	training's auc: 0.767492	valid_1's auc: 0.683906
[3000]	training's auc: 0.768635	valid_1's auc: 0.683713
[3050]	training's auc: 0.769401	valid_1's auc: 0.683635
[3100]	training's auc: 0.770378	valid_1's auc: 0.683805
[3150]	training's auc: 0.771192	valid_1's auc: 0.683836
Early stopping, best iteration is:
[2956]	training's auc: 0.767648	valid_1's auc: 0.683917
Fold  9 AUC : 0.683910
Train Index: [     1      2      4 ... 296234 296235 296236] ,Val Index: [     0      3     17 ... 296172 296200 296204]
Training until validation scores don't improve for 200 rounds.
[50]	training's auc: 0.670559	valid_1's auc: 0.654031
[100]	training's auc: 0.674388	valid_1's auc: 0.656911
[150]	training's auc: 0.677485	valid_1's auc: 0.658863
[200]	training's auc: 0.680136	valid_1's auc: 0.659953
[250]	training's auc: 0.682939	valid_1's auc: 0.661325
[300]	training's auc: 0.685708	valid_1

In [8]:
def nanMapping(x):
    if np.isnan(x):
        return 0
    else:
        return x

app_test = pd.read_csv('testing-set.csv', usecols=['order_id'])
predF=pd.DataFrame({"order_id":rid, "deal_or_not":sub_preds})
preds = pd.DataFrame({"order_id":app_test['order_id']})
preds=preds.merge(predF, on="order_id",how="outer")
preds['deal_or_not'] = preds.deal_or_not.apply(lambda x: nanMapping(x))
preds.to_csv("output/lgb_dart_" + str(roc_auc_score(Y, oof_preds)) + ".csv", index=False)

In [ ]:
from sklearn.linear_model import Lasso, Ridge, RidgeCV, ElasticNet
import xgboost as xgb
import lightgbm as lgb
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from catboost import Pool, CatBoostRegressor, cv


len1 = len(Y)
tind = np.zeros(len1, np.int)
for i in range(len1):
    tind[i]=i
import random as rn
rn.Random(4).shuffle(tind)
print(train_x.shape)

train_x, train_y = X[tind[1000:],:], Y[tind[1000:]]
valid_x, valid_y = X[tind[:1000],:], Y[tind[:1000]]

model_ridge = Ridge(alpha = 5)
model_ridge.fit(train_x, train_y)

model_lasso = Lasso(alpha = 0.005)
model_lasso.fit(train_x, train_y)

model_en = ElasticNet(alpha = 0.005)
model_en.fit(train_x, train_y)

model_gbr = GradientBoostingRegressor(n_estimators=1200, 
                                      learning_rate=0.05,
                                      max_depth=4, 
                                      max_features='sqrt',
                                      min_samples_leaf=15, 
                                      min_samples_split=10, 
                                      loss='huber',
                                      random_state=5)
model_gbr.fit(train_x, train_y)

model_xgb = xgb.XGBRegressor(colsample_bytree=0.2,
                             learning_rate=0.06,
                             max_depth=3,
                             n_estimators=1150)
model_xgb(train_x, train_y)


params = {
'nthread': 8, 'boosting_type': 'dart','objective': 'regression', 'metric': 'auc', 
'learning_rate': 0.01, 'num_leaves': 70,
'max_depth': 9, 'subsample': 0.9, 'feature_fraction': 0.9, 
'min_split_gain': 0.09, 'min_child_weight': 9.5,
'drop_rate':0.5, 'skip_drop':0.5, 'max_drop':5, 'uniform_drop':False, 
'xgboost_dart_mode':True, 'drop_seed':5 }

dtrain = lgb.Dataset(train_x, label=train_y)
dval = lgb.Dataset(valid_x, label=valid_y, reference=dtrain)
bst = lgb.train(params, dtrain, num_boost_round=15000, valid_sets=[dval], early_stopping_rounds=2500, 
                verbose_eval=500)

tmp_valid = bst.predict(valid_x, num_iteration=bst.best_iteration)
sub_preds = bst.predict(rx, num_iteration=bst.best_iteration) 

# Make the feature importance dataframe
print('AUC : %.6f' % (roc_auc_score(valid_y, tmp_valid)))
def nanMapping(x):
    if np.isnan(x):
        return 0
    else:
        return x

app_test = pd.read_csv('testing-set.csv', usecols=['order_id'])
predF=pd.DataFrame({"order_id":rid, "deal_or_not":sub_preds})
preds = pd.DataFrame({"order_id":app_test['order_id']})
preds=preds.merge(predF, on="order_id",how="outer")
preds['deal_or_not'] = preds.deal_or_not.apply(lambda x: nanMapping(x))
preds.to_csv("output/lgb_dart_" + str(roc_auc_score(valid_y, tmp_valid)) + ".csv", index=False)

In [ ]:
preds['deal_or_not']